In [1]:
import pandas as pd
import ast

In [ ]:
path="/srv/chawak/planning-with-llms/results/rl/training/"
#date="17_06/"
date='debug-2006/'
df= pd.read_csv(f'{path}{date}metrics.csv')
#df=pd.read_csv(f"{path}{inference_dir}")
df=df.drop(columns=['Unnamed: 0'])

In [ ]:
df['Terminate']=df['Terminate'].apply(ast.literal_eval)

In [ ]:
def terminate_per_problem(lst, chunk_size):
    return [sum(lst[i:i + chunk_size]) for i in range(0, len(lst), chunk_size)]

In [ ]:
import yaml
with open("/srv/chawak/planning-with-llms/src/rl/config.yaml", "r") as f:
    cfg=yaml.safe_load(f)
group_size=cfg['training']['num_generations']

In [ ]:
df['Terminate_chunks'] = df['Terminate'].apply(lambda x: terminate_per_problem(x, group_size))

In [ ]:
df['Terminate_chunks']

In [ ]:
# transposed

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Data
metrics = ['reward/format_reward', 'reward/plan_reward', 'reward/bonus_reward', '# Terminate']
epochs = np.arange(len(df))
bar_width = 0.2

plt.figure(figsize=(12, 6))

for i, metric in enumerate(metrics):
    bar_positions = epochs + i * bar_width
    values = df[metric]
    
    # Plot bars
    bars = plt.bar(bar_positions, values, width=bar_width, label=metric)
    
    # Add value labels on top
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, height + 0.3, f'{height:.1f}', 
                 ha='center', va='bottom', fontsize=8)

# X-axis setup
plt.xticks(epochs + bar_width * (len(metrics)-1)/2, [f'Epoch {i}' for i in epochs])
plt.xlabel('Epoch')
plt.ylabel('Value')
plt.title('Epoch-wise rewards and #plans that reach goal')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Data
metrics = ['reward/format_reward', 'reward/plan_reward', 'reward/bonus_reward', '# Terminate']
epochs = np.arange(len(df))

plt.figure(figsize=(12, 6))

# Plot a separate line for each metric
for metric in metrics:
    values = df[metric]
    plt.plot(epochs, values, marker='o', label=metric)

    # Add value labels at each point
    for x, y in zip(epochs, values):
        plt.text(x, y + 0.2, f'{y:.1f}', ha='center', va='bottom', fontsize=8)

    #moving-average for plan-reward
    if metric=="reward/plan_reward":
        rolling_values=df[metric].rolling(window=3,min_periods=1).mean()
        plt.plot(epochs,rolling_values,linestyle='--',linewidth=2, label= 'reward/plan_reward(rolling avg of 3)')

# X-axis setup
plt.xticks(epochs, [f'Epoch {i}' for i in epochs])
plt.xlabel('Epoch')
plt.ylabel('Value')
plt.title('Epoch-wise rewards and #plans that reach goal')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


scrap for fun

In [2]:
import regex as re

In [36]:
gibberish=r"(.*)"
plan=r"\[PLAN\](.*?)\[PLAN END\]"
think=r"<think>(.*?)<\/think>"

In [86]:
#case-1: no gibberish, no plan, yes think 
pattern1=f"{think}"
compiled1=re.compile(pattern1,re.DOTALL)

#case-2: no gibberish, yes plan, no think
pattern2=f"{plan}"
compiled2=re.compile(pattern2,re.DOTALL)

#case-3: no gibberish, yes plan, yes think
pattern3=f"{think}\s*{plan}"
compiled3=re.compile(pattern3,re.DOTALL)

#case-6: yes gibberish, yes plan, yes think
pattern6=f"{think}{gibberish}{plan}"
compiled6=re.compile(pattern6,re.DOTALL)


In [98]:
def reward_func(response):
    
    response=response.strip()
    score=0

    #ONLY think tag
    fullmatchthink=False
    if compiled1.fullmatch(response):
        score=7
        fullmatchthink=True
    if compiled1.search(response) and not fullmatchthink:
        score=5
    
    #ONLY plan tag
    fullmatchplan=False
    if compiled2.fullmatch(response):
        score=3
        fullmatchplan=True
    if compiled2.search(response) and not fullmatchplan:
        score=2
    
    #COMBO of both tags
    fullmatchcombo=False
    if compiled3.fullmatch(response):
        score=20
        fullmatchcombo=True
    if compiled6.search(response) and not fullmatchcombo:
        score=15

    return score

In [44]:
response='''<think>
\nFirst unstack the green block from on top of the red block.\n
Then put down the green block.\nThen pick up the red block.\nThen stack the red block on top of the green block.
\nThen pick up the pink block.\nThen stack the pink block on top of the red block.\n
</think>
\n[PLAN]
\nunstack the green block from on top of the red block\nput down the green block
\npick up the red block\nstack the red block on top of the green block\npick up the pink block
\nstack the pink block on top of the red block\n
[PLAN END]\n'''

In [96]:
response='''<think>
\nFirst unstack the green block from on top of the red block.\n
Then put down the green block.\nThen pick up the red block.\nThen stack the red block on top of the green block.
\nThen pick up the pink block.\nThen stack the pink block on top of the red block.\n
</think>
\n[PLAN]
\nunstack the green block from on top of the red block\nput down the green block
\npick up the red block\nstack the red block on top of the green block\npick up the pink block
\nstack the pink block on top of the red block\n
[PLAN END]\n
'''

In [97]:
reward_func(response)

True


20

In [ ]:
import inspect
from unsloth import FastLanguageModel

print(inspect.signature(FastLanguageModel.from_pretrained))

In [ ]:
print('<bos><start_of_turn>user\nI am a blocksworld plan generator.\nI first think about the reasoning process in the mind and then provide the user with the plan.\nThe reasoning process and plan are enclosed within <think> </think> and [PLAN] [PLAN END] tags, respectively,\ni.e., <think> reasoning process here </think> [PLAN] plan here [PLAN END].\n\n\nI am playing with a set of blocks where I need to arrange the blocks into stacks\n        Here are the actions I can do: Pick up a block, Unstack a block from on top of another block, Put down a block, Stack a block on top of another block.\n        I have the following restrictions on my actions:\n        I can only pick up or unstack one block at a time\n        I can only pick up or unstack a block if my hand is empty\n        I can only pick up a block if the block is on the table and the block is clear\n        A block is clear if the block has no other blocks on top of it and if the block is not picked up\n        I can only unstack a block from on top of another block if the block I am unstacking was really on top of the other block\n        I can only unstack a block from on top of another block if the block I am unstacking is clear\n        Once I pick up or unstack a block, I am holding the block\n        I can only put down a block that I am holding\n        I can only stack a block on top of another block if I am holding the block being stacked\n        I can only stack a block on top of another block if the block onto which I am stacking the block is clear\n        Once I put down or stack a block, my hand becomes empty\n        Once you stack a block on top of a second block, the second block is no longer clear\n\n[STATEMENT]\nAs initial conditions I have that, the brown block is clear, the teal block is clear,  the hand is empty, the violet block is on the table, the brown block is on top of the violet block, the teal block is on the table.\nMy goal is to have that  the brown block is on the table, the violet block is on top of the brown block, the teal block is on top of the violet block.\n\nMy plan is as follows:\n\n[PLAN]\nunstack the brown block from on top of the violet block\nput down the brown block\npick up the violet block\nstack the violet block on top of the brown block\npick up the teal block\nstack the teal block on top of the violet block\n[PLAN END]\n\n[STATEMENT]\nAs initial conditions I have that, the green block is clear,  the hand is empty, the pink block is on the table, the red block is on top of the pink block, the green block is on top of the red block.\nMy goal is to have that  the pink block is on the table, the red block is on top of the pink block, the green block is on the table.\n\nMy plan is as follows: \n\n[PLAN]<end_of_turn>\n<start_of_turn>model\n')